<a href="https://colab.research.google.com/github/laisab/IC/blob/main/rascunho_ic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tabulate import tabulate

In [52]:
# Dicionário com datasets e seus respectivos targets
dicio_datasets = {
    'Adult.csv': 'income',
    'Biomed.csv': 'class',
    'COMPAS.csv': 'two_year_recid',
    'ContraceptiveMethodChoice.csv': "'Contraceptive_method_used'",
    'DefaultOfCreditCardClients.csv': 'default.payment.next.month',
    'EstimationOfObesity.csv': 'NObeyesdad',
    'GermanCreditRisk.csv': "class'",
    'HabermansSurvivalData.csv': "'Survival_status'",
    'HeartDiseaseStatlog.csv': 'target',
    'IndianLiverPatientDataset.csv': 'Class',
    'IrishEducationalTransitionsData.csv': 'Leaving_Certificate',
    'LowBirthWeightData.csv': 'binaryClass',
    'PimaIndiansDiabetesDatabase.csv': "'class'",
    'PopularKids.csv': 'Goals',
    'PortugueseBankMarketing.csv': 'Subscription',
    'Schizo.csv': 'class',
    'SocialMobility.csv': 'binaryClass',
    'ThyroidSicknessDetermination.csv': 'Class'
}

**Idade**
*   Menor de idade: < 18
*   Jovem: 18 aos 30 anos
*   Adulto: 31 aos 60 anos
*   Idoso: acima dos 60 anos


**Idade menor** (dataset PopularKids)
*  Criança: <= 9 anos
*  Pré-adolescente: 10 aos 13 anos
*  Adolescente: > 13 anos


In [64]:
idade = {
    'bins': [0, 18, 30, 60, np.inf],
    'labels': ['menor de idade', 'jovem', 'adulto', 'idoso']
}

idade_menor = {
    'bins': [9, 10, 13, np.inf],
    'labels': ['criança', 'pre-adolescente', 'adolescente']
}

In [65]:
# Dicionário com datasets e seus respectivos atributos sensíveis
dicio_atributos_sensiveis = {
    'Adult.csv': {
        'age': idade,
        'education': None,
        'marital.status': None,
        'relationship': None,
        'race': None,
        'sex': None,
        'native.country': None
    },

    'Biomed.csv': {
        'Age_of_patient': idade
    },

    'COMPAS.csv': {
      'sex': None,
      'age': idade
    },

    'ContraceptiveMethodChoice.csv': {
      "'Wifes_age'": idade,
      "'Wifes_education'": None,
      "'Husbands_education'": None,
      "'Wifes_religion'": None,
      "'Wifes_now_working%3F'": None,
      "'Husbands_occupation'": None,
      "'Standard-of-living_index'": None
    },

    'DefaultOfCreditCardClients.csv': {
      'SEX': None,
      'EDUCATION': None,
      'MARRIAGE': None,
      'AGE': idade
    },

    'EstimationOfObesity.csv': {
      'Gender': None,
      'Age': idade
    },

    'GermanCreditRisk.csv': {
      "personal_status'": None,
      "age'": idade,
      "housing'": None,
      "job'": None
    },

    'HabermansSurvivalData.csv': {
      "'Age_of_patient_at_time_of_operation'": idade
    },

    'HeartDiseaseStatlog.csv': {
      'age': idade,
      'sex': None
    },

    'IndianLiverPatientDataset.csv': {
      'V1': idade,
      'V2': None
    },

    'IrishEducationalTransitionsData.csv': {
      'Sex': None,
      'Prestige_score': None
    },

    'LowBirthWeightData.csv': {
      'AGE': idade,
      'RACE': None,
    },

    'PimaIndiansDiabetesDatabase.csv': {
      "'age'": idade,
    },

    'PopularKids.csv': {
      'Gender': None,
      'Age': idade_menor,
      'Race': None,
      'Urban/Rural': None
    },

    'PortugueseBankMarketing.csv': {
      'Age': idade,
      'Job': None,
      'Marital Status': None,
      'Education': None
    },

    'Schizo.csv': {
      'sex': None
    },

    'SocialMobility.csv': {
      'family_structure': None,
      'race': None
    },

    'ThyroidSicknessDetermination.csv': {
      'age': None,
      'sex': None
    }
}

In [ ]:
caminho_drive = '/content/drive/MyDrive/IC/Datasets/'
novo_caminho = '/content/drive/MyDrive/Teste/'

In [ ]:
def separar_feature_target(caminho_completo):

  try:
    df = pd.read_csv(caminho_completo)

    # Obtém o nome do target
    nome_arquivo = os.path.basename(caminho_completo)
    nome_target = dicio_datasets.get(nome_arquivo)

    if not nome_target:
      raise ValueError(f'Coluna target não encontrada para o arquivo {nome_arquivo}')

    # Separa features (X) e target (y)
    X = df.drop(columns=[nome_target])
    y = df[nome_target]

    # Dicionário para armazenar o mapeamento de colunas OneHotEncoder
    # Exemplo: {'sex': ['sex_female', 'sex_male']}
    colunas_onehot = {}

    # Codificação de features categóricas com OneHotEncoder
    colunas_categoricas = X.select_dtypes(include=['object']).columns

    # Verificação de colunas categóricas para evitar erros com datasets numéricos
    if colunas_categoricas.size > 0:
      # Lida com categorias desconhecidas em dados futuros
      enc = OneHotEncoder(handle_unknown='ignore')
      X_encoded = enc.fit_transform(X[colunas_categoricas]).toarray()

      # Nomes das novas features criadas pelo OneHotEncoder
      novo_nome_X = enc.get_feature_names_out(colunas_categoricas)
      X_encoded_df = pd.DataFrame(X_encoded, columns=novo_nome_X)

      # Para cada coluna categórica original, a lista das novas colunas são armazenadas
      for coluna_original in colunas_categoricas:
        colunas_onehot[coluna_original] = [nome for nome in novo_nome_X if nome.startswith(f'{coluna_original}_')]

      # Remove as colunas categóricas originais
      X = X.select_dtypes(exclude=['object'])
      X = pd.concat([X, X_encoded_df], axis=1)

    return X, y, colunas_onehot

  except Exception as e:
    print(f'\nErro ao separar os dados de {nome_arquivo}: {e}')
    return None

In [ ]:
# Predição dos datasets
def predizer_dataset(modelo, X, y, cv):
  try:
    y_pred = cross_val_predict(modelo, X, y, cv=cv, method='predict')
    y_prob = cross_val_predict(modelo, X, y, cv=cv, method='predict_proba')
    return y_pred, y_prob

  except Exception as e:
    print(f'\nErro ao predizer dataset: {e}')
    return None

In [71]:
# Avaliação dos datasets
def avaliar_dataset(y, y_pred, y_prob):
  try:
    metricas = {
      'accuracy': accuracy_score,
      'f1_score': lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro'),
      'roc_auc_score': lambda y_true, y_pred_proba: roc_auc_score(y_true, y_pred_proba, average='macro', multi_class='ovr') if len(np.unique(y_true)) > 2 else (roc_auc_score(y_true, y_pred_proba[:, 1]) if len(np.unique(y_true)) == 2 else np.nan)
    }

    avaliacao = {}
    for metrica, metrica_funcao in metricas.items():
      try:
        if metrica == 'roc_auc_score':
          if len(np.unique(y)) < 2:
            avaliacao[metrica] = np.nan
          else:
            avaliacao[metrica] = metrica_funcao(y, y_prob)
        else:
          avaliacao[metrica] = metrica_funcao(y, y_pred)
      except Exception as e:
        print(f'\nErro ao calcular {metrica}: {e}')
        avaliacao[metrica] = None

    return avaliacao

  except Exception as e:
    print(f'\nErro ao avaliar dataset: {e}')
    return None

In [68]:
# Avaliação dos segmentos
def avaliar_segmentos(X, y, y_pred, y_prob, X_onehot, avaliar_dataset_func, dataset="", atr_sensivel=None, num_bins=5):
  """
  Args:
    X: features
    y: target
    y_pred: predições
    y_prob: probabilidades
    avaliar_dataset_func: função avaliar_dataset
    dataset: nome do dataset
    atributos_sensiveis: dicionário com atributos sensíveis
    bins: número de segmentos para atributos numéricos sem label (padrão = 5)
  """

  res_segmentos = []

  for coluna_original, bin_labels in atr_sensivel.items():
    if coluna_original not in X.columns and coluna_original not in X_onehot:
      print(f'  Atributo {coluna_original} não encontrado para o dataset {dataset}')
      continue

    print(f'\n  Avaliação dos segmentos para o atributo {coluna_original} do dataset {dataset}')

    # Atributos OneHotEncoder
    if coluna_original in X_onehot:
      colunas_onehot = X_onehot[coluna_original]

      for coluna_onehot in colunas_onehot:
        idx = X[coluna_onehot] == 1

        if idx.sum() > 0:
          subset_y = y[idx]
          subset_y_pred = y_pred[idx]
          subset_y_prob = y_prob[idx]
          nome_categorico = coluna_onehot.replace(f'{coluna_original}_', '')
          print(f'  Segmento {nome_categorico}')
          avaliacao = avaliar_dataset_func(subset_y, subset_y_pred, subset_y_prob)
          print(avaliacao)
        else:
          nome_categorico = coluna_onehot.replace(f'{coluna_original}_', '')
          print(f'  Segmento {nome_categorico} sem dados')

    # Atributos sem OneHotEncoder (numéricos ou categóricos que não passaram pelo OneHot)
    else:
      coluna_dataset = X[coluna_original]
      bin_col = coluna_dataset

      # Bins configurados
      if bin_labels is not None and isinstance(bin_labels, dict) and 'bins' in bin_labels and 'labels' in bin_labels:
        try:
          bin_col = pd.cut(coluna_dataset, bins=bin_labels['bins'], labels=bin_labels['labels'], include_lowest=True, right=True)
        except Exception as e:
          print(f'  Erro ao configurar os bins para o atributo {coluna_original}: {e}')
          bin_col = coluna_dataset
          continue

      # Bins automáticos
      elif coluna_dataset.dtype != 'object' and len(coluna_dataset.unique()) > num_bins * 2:
        try:
          bin_col = pd.qcut(coluna_dataset, q=num_bins, duplicates='drop')
        except Exception as e:
          print(f'  Erro ao configurar os bins para o atributo {coluna_original}: {e}')
          bin_col = coluna_dataset
          continue

      valor_unico = bin_col.unique().tolist()

      try:
        valor_unico.sort(key=lambda x: x.left if isinstance(x, pd.Interval) else x)
      except TypeError:
        try:
          valor_unico.sort(key=str)
        except TypeError:
          pass

      for valor in valor_unico:
        idx = bin_col == valor

        if idx.sum() > 0:
          subset_y = y[idx]
          subset_y_pred = y_pred[idx]
          subset_y_prob = y_prob[idx]
          nome_grupo = str(valor)
          print(f'  Segmento {nome_grupo}')
          avaliacao = avaliar_dataset_func(subset_y, subset_y_pred, subset_y_prob)
          print(avaliacao)
        else:
          nome_grupo = str(valor)
          print(f'  Segmento {nome_grupo} sem dados')

In [69]:
# Lista os arquivos csv na pasta
arquivos = [f for f in os.listdir(caminho_drive) if f.endswith('.csv') and f in dicio_datasets]
arquivos

['IndianLiverPatientDataset.csv',
 'HabermansSurvivalData.csv',
 'GermanCreditRisk.csv',
 'PortugueseBankMarketing.csv',
 'EstimationOfObesity.csv',
 'DefaultOfCreditCardClients.csv',
 'Adult.csv',
 'COMPAS.csv',
 'ContraceptiveMethodChoice.csv',
 'ThyroidSicknessDetermination.csv',
 'HeartDiseaseStatlog.csv',
 'SocialMobility.csv',
 'LowBirthWeightData.csv',
 'PopularKids.csv',
 'IrishEducationalTransitionsData.csv',
 'Biomed.csv',
 'Schizo.csv',
 'PimaIndiansDiabetesDatabase.csv']

In [72]:
# Processamento de cada dataset
resultados_dataset = []
res_metrica_tabela = []
res_segmentos_tabela = []

modelo = RandomForestClassifier(n_estimators=20, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for arquivo in arquivos:
  caminho_completo = os.path.join(caminho_drive, arquivo)
  dados = separar_feature_target(caminho_completo)

  if dados is not None:
    X, y, X_onehot = dados
    predicao, probabilidade = predizer_dataset(modelo, X, y, cv)
    resultados_dataset = avaliar_dataset(y, predicao, probabilidade)
    print(f'\n  Resultados do dataset {arquivo}: {resultados_dataset}')

    res_metrica_tabela.append([
      arquivo,
      modelo,
      resultados_dataset.get('accuracy'),
      resultados_dataset.get('f1_score'),
      resultados_dataset.get('roc_auc_score')
    ])

    resultados_segmentos = avaliar_segmentos(X, y, predicao, probabilidade, X_onehot, avaliar_dataset, arquivo, dicio_atributos_sensiveis.get(arquivo, {}), 5)

    arquivo_pred = f"Predicao_{arquivo}"
    caminho_pred = os.path.join(novo_caminho, arquivo_pred)
    pd.DataFrame(predicao, columns=[arquivo]).to_csv(caminho_pred, index=False)

    arquivo_segmento = f"Segmentos_{arquivo}"
    caminho_segmento = os.path.join(novo_caminho, arquivo_segmento)
    pd.DataFrame(res_segmentos_tabela).to_csv(caminho_segmento, index=False)
  else:
    print(f'Não foi possível processar o dataset {arquivo}')


  Resultados do dataset IndianLiverPatientDataset.csv: {'accuracy': 0.7066895368782161, 'f1_score': 0.5756173562127256, 'roc_auc_score': np.float64(0.7446813104560109)}

  Avaliação dos segmentos para o atributo V1 do dataset IndianLiverPatientDataset.csv
  Segmento adulto
{'accuracy': 0.7317073170731707, 'f1_score': 0.5460013670539986, 'roc_auc_score': np.float64(0.7400882825040128)}
  Segmento idoso
{'accuracy': 0.7474747474747475, 'f1_score': 0.6008708272859217, 'roc_auc_score': np.float64(0.7546296296296297)}
  Segmento jovem
{'accuracy': 0.5949367088607594, 'f1_score': 0.5641379310344827, 'roc_auc_score': np.float64(0.6686046511627908)}
  Segmento menor de idade
{'accuracy': 0.5833333333333334, 'f1_score': 0.5669607056936649, 'roc_auc_score': np.float64(0.7238095238095238)}

  Avaliação dos segmentos para o atributo V2 do dataset IndianLiverPatientDataset.csv
  Segmento Female
{'accuracy': 0.6197183098591549, 'f1_score': 0.5301470588235294, 'roc_auc_score': np.float64(0.676739130

In [ ]:
colunas = ['NOME DO DATASET', 'MODELO', 'ACURÁCIA', 'F1', 'AUC']
print(tabulate(res_metrica_tabela, headers=colunas, tablefmt='fancy_grid'))

╒═════════════════════════════════════╤══════════════════════════════════════════════════════════╤════════════╤══════════╤══════════╕
│ NOME DO DATASET                     │ MODELO                                                   │   ACURÁCIA │       F1 │      AUC │
╞═════════════════════════════════════╪══════════════════════════════════════════════════════════╪════════════╪══════════╪══════════╡
│ IndianLiverPatientDataset.csv       │ RandomForestClassifier(n_estimators=20, random_state=42) │   0.70669  │ 0.575617 │ 0.744681 │
├─────────────────────────────────────┼──────────────────────────────────────────────────────────┼────────────┼──────────┼──────────┤
│ HabermansSurvivalData.csv           │ RandomForestClassifier(n_estimators=20, random_state=42) │   0.699346 │ 0.565126 │ 0.620988 │
├─────────────────────────────────────┼──────────────────────────────────────────────────────────┼────────────┼──────────┼──────────┤
│ GermanCreditRisk.csv                │ RandomForestClassifier

In [ ]:
"""
# Colunas categóricas
if any(col.startswith('V2_') for col in X.columns):
  idx = X["V2_Female"] == True
  print(avaliar_dataset(y[idx], predicao[idx], probabilidade[idx]))
  idx = X["V2_Male"] == True
  print(avaliar_dataset(y[idx], predicao[idx], probabilidade[idx]))
else:
  print(f"Coluna não encontrada")

idx = X['sex'] == 'male'
avaliar_dataset(y[idx], predicao[idx], probabilidade[idx])
idx = X['sex'] == 'female'
avaliar_dataset(y[idx], predicao[idx], probabilidade[idx])
"""

'\n# Colunas categóricas\nif any(col.startswith(\'V2_\') for col in X.columns):\n  idx = X["V2_Female"] == True\n  print(avaliar_dataset(y[idx], predicao[idx], probabilidade[idx]))\n  idx = X["V2_Male"] == True\n  print(avaliar_dataset(y[idx], predicao[idx], probabilidade[idx]))\nelse:\n  print(f"Coluna não encontrada")\n\nidx = X[\'sex\'] == \'male\'\navaliar_dataset(y[idx], predicao[idx], probabilidade[idx])\nidx = X[\'sex\'] == \'female\'\navaliar_dataset(y[idx], predicao[idx], probabilidade[idx])\n'